# 경사하강법을 이용한 얕은 신경망 실습

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
EPOCHS = 1000

## 네트워크 구조 정의

### 얕은 신경망
- 입력 계층 : 2 , 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [15]:
class MyModel(tf.keras.Model) :
    def __init__(self) :
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid') # 인풋 2차원, Dense = fully-connected layer, 128층
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [23]:
@tf.function # 오토 그래프?
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape :
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables) # grad(loss), df(x)/dx
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    
    train_metric(labels, predictions)

## 데이터셋 생성, 전처리

In [24]:
np.random.seed(0) # 랜덤 값을 고정한다

pts = list() #입력값
labels = list() #정답
center_pts = np.random.uniform(-8.0, 8.0, (10, 2)) # 10개 2차원
for label, center_pt in enumerate(center_pts) :
    for _ in range(100) :
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
        
pts = np.stack(pts, axis=0).astype(np.float32) # GPU연산 -> float32로 구성
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## 모델 생성

In [25]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
- CrossEntropy, Adam Optimizer

In [26]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
- Accuracy

In [27]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [29]:
for epoch in range(EPOCHS) :
    for x, label in train_ds :
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss : {}, Accuracy : {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100))

Epoch 1, Loss : 0.2966020405292511, Accuracy : 88.80387878417969
Epoch 2, Loss : 0.2965625524520874, Accuracy : 88.8045883178711
Epoch 3, Loss : 0.2964973449707031, Accuracy : 88.8052978515625
Epoch 4, Loss : 0.29642587900161743, Accuracy : 88.80585479736328
Epoch 5, Loss : 0.29637202620506287, Accuracy : 88.80741119384766
Epoch 6, Loss : 0.29630976915359497, Accuracy : 88.8088150024414
Epoch 7, Loss : 0.2962495684623718, Accuracy : 88.8099365234375
Epoch 8, Loss : 0.29618099331855774, Accuracy : 88.8110580444336
Epoch 9, Loss : 0.2961167097091675, Accuracy : 88.8118896484375
Epoch 10, Loss : 0.29604706168174744, Accuracy : 88.81356048583984
Epoch 11, Loss : 0.2959926128387451, Accuracy : 88.81481170654297
Epoch 12, Loss : 0.29593372344970703, Accuracy : 88.81605529785156
Epoch 13, Loss : 0.29587891697883606, Accuracy : 88.81715393066406
Epoch 14, Loss : 0.2958095669746399, Accuracy : 88.81867218017578
Epoch 15, Loss : 0.29575127363204956, Accuracy : 88.81935119628906
Epoch 16, Loss : 

Epoch 127, Loss : 0.2896226942539215, Accuracy : 88.95267486572266
Epoch 128, Loss : 0.2895670235157013, Accuracy : 88.95417785644531
Epoch 129, Loss : 0.2895185947418213, Accuracy : 88.955322265625
Epoch 130, Loss : 0.2894669473171234, Accuracy : 88.95681762695312
Epoch 131, Loss : 0.28942784667015076, Accuracy : 88.95747375488281
Epoch 132, Loss : 0.289381742477417, Accuracy : 88.9583740234375
Epoch 133, Loss : 0.28933554887771606, Accuracy : 88.95962524414062
Epoch 134, Loss : 0.289282888174057, Accuracy : 88.96099090576172
Epoch 135, Loss : 0.2892366945743561, Accuracy : 88.96212005615234
Epoch 136, Loss : 0.28918805718421936, Accuracy : 88.96395874023438
Epoch 137, Loss : 0.2891342341899872, Accuracy : 88.96508026123047
Epoch 138, Loss : 0.2890889048576355, Accuracy : 88.966552734375
Epoch 139, Loss : 0.2890385091304779, Accuracy : 88.96719360351562
Epoch 140, Loss : 0.28898555040359497, Accuracy : 88.96913146972656
Epoch 141, Loss : 0.288930743932724, Accuracy : 88.97000885009766

Epoch 256, Loss : 0.28394800424575806, Accuracy : 89.0855484008789
Epoch 257, Loss : 0.28390181064605713, Accuracy : 89.08682250976562
Epoch 258, Loss : 0.2838647961616516, Accuracy : 89.0884017944336
Epoch 259, Loss : 0.2838241755962372, Accuracy : 89.08882904052734
Epoch 260, Loss : 0.2837797999382019, Accuracy : 89.08988189697266
Epoch 261, Loss : 0.28373900055885315, Accuracy : 89.0910415649414
Epoch 262, Loss : 0.28370237350463867, Accuracy : 89.09188079833984
Epoch 263, Loss : 0.28365954756736755, Accuracy : 89.09292602539062
Epoch 264, Loss : 0.2836202383041382, Accuracy : 89.09407806396484
Epoch 265, Loss : 0.28358808159828186, Accuracy : 89.09542846679688
Epoch 266, Loss : 0.28354910016059875, Accuracy : 89.09554290771484
Epoch 267, Loss : 0.28351086378097534, Accuracy : 89.09679412841797
Epoch 268, Loss : 0.283475399017334, Accuracy : 89.097412109375
Epoch 269, Loss : 0.2834353744983673, Accuracy : 89.09803771972656
Epoch 270, Loss : 0.2833922207355499, Accuracy : 89.09927368

Epoch 381, Loss : 0.27958276867866516, Accuracy : 89.19971466064453
Epoch 382, Loss : 0.2795538902282715, Accuracy : 89.20063781738281
Epoch 383, Loss : 0.27952298521995544, Accuracy : 89.20175170898438
Epoch 384, Loss : 0.27949258685112, Accuracy : 89.20267486572266
Epoch 385, Loss : 0.27946820855140686, Accuracy : 89.20368194580078
Epoch 386, Loss : 0.27943363785743713, Accuracy : 89.20459747314453
Epoch 387, Loss : 0.2794046103954315, Accuracy : 89.20570373535156
Epoch 388, Loss : 0.27937600016593933, Accuracy : 89.20689392089844
Epoch 389, Loss : 0.27935320138931274, Accuracy : 89.20817565917969
Epoch 390, Loss : 0.279316782951355, Accuracy : 89.20917510986328
Epoch 391, Loss : 0.27928367257118225, Accuracy : 89.21026611328125
Epoch 392, Loss : 0.2792498767375946, Accuracy : 89.21126556396484
Epoch 393, Loss : 0.27922239899635315, Accuracy : 89.21234893798828
Epoch 394, Loss : 0.2791915535926819, Accuracy : 89.21316528320312
Epoch 395, Loss : 0.27916207909584045, Accuracy : 89.2141

Epoch 505, Loss : 0.27600687742233276, Accuracy : 89.30696105957031
Epoch 506, Loss : 0.275976300239563, Accuracy : 89.30762481689453
Epoch 507, Loss : 0.2759435474872589, Accuracy : 89.30853271484375
Epoch 508, Loss : 0.2759139835834503, Accuracy : 89.30926513671875
Epoch 509, Loss : 0.27588599920272827, Accuracy : 89.31000518798828
Epoch 510, Loss : 0.27586206793785095, Accuracy : 89.31066131591797
Epoch 511, Loss : 0.2758385241031647, Accuracy : 89.31172180175781
Epoch 512, Loss : 0.2758108973503113, Accuracy : 89.31253814697266
Epoch 513, Loss : 0.2757841944694519, Accuracy : 89.31343841552734
Epoch 514, Loss : 0.27575433254241943, Accuracy : 89.3143310546875
Epoch 515, Loss : 0.27572229504585266, Accuracy : 89.31513977050781
Epoch 516, Loss : 0.27569520473480225, Accuracy : 89.31553649902344
Epoch 517, Loss : 0.27566611766815186, Accuracy : 89.31626892089844
Epoch 518, Loss : 0.275637686252594, Accuracy : 89.3169174194336
Epoch 519, Loss : 0.27561378479003906, Accuracy : 89.317550

Epoch 629, Loss : 0.27293989062309265, Accuracy : 89.39539337158203
Epoch 630, Loss : 0.27292075753211975, Accuracy : 89.39636993408203
Epoch 631, Loss : 0.2728999853134155, Accuracy : 89.39689636230469
Epoch 632, Loss : 0.2728744447231293, Accuracy : 89.39749908447266
Epoch 633, Loss : 0.27284783124923706, Accuracy : 89.39833068847656
Epoch 634, Loss : 0.27282336354255676, Accuracy : 89.39900970458984
Epoch 635, Loss : 0.27280545234680176, Accuracy : 89.3995361328125
Epoch 636, Loss : 0.27278807759284973, Accuracy : 89.40020751953125
Epoch 637, Loss : 0.272767573595047, Accuracy : 89.40103149414062
Epoch 638, Loss : 0.27274271845817566, Accuracy : 89.40162658691406
Epoch 639, Loss : 0.2727166712284088, Accuracy : 89.40192413330078
Epoch 640, Loss : 0.2726902961730957, Accuracy : 89.40259552001953
Epoch 641, Loss : 0.2726697325706482, Accuracy : 89.4033432006836
Epoch 642, Loss : 0.2726503312587738, Accuracy : 89.40363311767578
Epoch 643, Loss : 0.272627592086792, Accuracy : 89.4040069

Epoch 756, Loss : 0.2702043056488037, Accuracy : 89.47737121582031
Epoch 757, Loss : 0.2701844871044159, Accuracy : 89.47779846191406
Epoch 758, Loss : 0.27015939354896545, Accuracy : 89.47843170166016
Epoch 759, Loss : 0.2701367139816284, Accuracy : 89.47872161865234
Epoch 760, Loss : 0.2701135277748108, Accuracy : 89.47948455810547
Epoch 761, Loss : 0.27009472250938416, Accuracy : 89.48004913330078
Epoch 762, Loss : 0.2700730562210083, Accuracy : 89.48074340820312
Epoch 763, Loss : 0.27005118131637573, Accuracy : 89.48137664794922
Epoch 764, Loss : 0.2700285017490387, Accuracy : 89.4822769165039
Epoch 765, Loss : 0.2700064182281494, Accuracy : 89.4827651977539
Epoch 766, Loss : 0.26998835802078247, Accuracy : 89.48332977294922
Epoch 767, Loss : 0.269965797662735, Accuracy : 89.48401641845703
Epoch 768, Loss : 0.2699441611766815, Accuracy : 89.4846420288086
Epoch 769, Loss : 0.2699258327484131, Accuracy : 89.48526763916016
Epoch 770, Loss : 0.2699134349822998, Accuracy : 89.4858932495

Epoch 883, Loss : 0.2678092420101166, Accuracy : 89.54853057861328
Epoch 884, Loss : 0.2677890360355377, Accuracy : 89.54925537109375
Epoch 885, Loss : 0.26777225732803345, Accuracy : 89.54947662353516
Epoch 886, Loss : 0.26775193214416504, Accuracy : 89.55007934570312
Epoch 887, Loss : 0.2677346467971802, Accuracy : 89.55067443847656
Epoch 888, Loss : 0.267715722322464, Accuracy : 89.55108642578125
Epoch 889, Loss : 0.2676991820335388, Accuracy : 89.55142974853516
Epoch 890, Loss : 0.2676790952682495, Accuracy : 89.55183410644531
Epoch 891, Loss : 0.26765820384025574, Accuracy : 89.55230712890625
Epoch 892, Loss : 0.26764559745788574, Accuracy : 89.55278015136719
Epoch 893, Loss : 0.267627090215683, Accuracy : 89.55318450927734
Epoch 894, Loss : 0.2676073908805847, Accuracy : 89.55396270751953
Epoch 895, Loss : 0.2675880491733551, Accuracy : 89.55474853515625
Epoch 896, Loss : 0.2675783336162567, Accuracy : 89.5551528930664
Epoch 897, Loss : 0.26756203174591064, Accuracy : 89.55567932

## 데이터셋 및 학습 파라미터 저장

In [30]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights() # 은닉층 가중치와 편향
W_o, b_o = model.d2.get_weights() # 출력층 가중치와 편향
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz', W_h=W_h, b_h=b_h, W_o=W_o, b_o=b_o)